## Thuật toán sử dụng công thức sterlin để giải đa thức nội suy 

In [1]:
import numpy as np
import sys 

# Yêu cầu NumPy không bao giờ rút gọn (sử dụng giá trị lớn nhất có thể)
np.set_printoptions(threshold=sys.maxsize)

# ... (phần code tính toán và in kết quả của bạn) ...
# Ví dụ:
# print("Kết quả vector chẵn mới (TRƯỚC KHI ĐẢO NGƯỢC):", new_even_coeffs_calc)

In [2]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_and_validate_gauss_data(file_x_path, file_y_path):
    """Hàm đọc dữ liệu từ file và kiểm tra các điều kiện cho nội suy Gauss."""
    try:
        x_coords = np.loadtxt(file_x_path, ndmin=1).astype(float)
        y_coords = np.loadtxt(file_y_path, ndmin=1).astype(float)
    except (FileNotFoundError, ValueError):
        raise IOError("LỖI: Không tìm thấy file hoặc file chứa dữ liệu không hợp lệ.")

    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")

    if len(np.unique(x_coords)) != len(x_coords):
        raise ValueError("LỖI: Các giá trị x không được trùng lặp.")
        
    # Kiểm tra điều kiện quan trọng: các mốc x phải cách đều
    diffs = np.diff(x_coords)
    if not np.allclose(diffs, diffs[0]):
        raise ValueError("LỖI: Các mốc nội suy x phải cách đều nhau để áp dụng công thức Gauss.")
        
    return x_coords, y_coords

# --- Chương trình chính cho Ô 1 ---
try:
    x_coords, y_coords = load_and_validate_gauss_data('FileX.txt', 'FileY.txt')
    print("Dữ liệu hợp lệ và đã được đọc thành công!")
    print("-" * 50)
    print("Các mốc nội suy x:", x_coords.tolist())
    print("Các giá trị y tương ứng:", y_coords.tolist())
except (IOError, ValueError) as e:
    print(str(e))


Dữ liệu hợp lệ và đã được đọc thành công!
--------------------------------------------------
Các mốc nội suy x: [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]
Các giá trị y tương ứng: [0.524, -0.084, -0.964, -2.121, -3.558, -5.272, -7.255, -9.494, -11.973]


In [3]:
import numpy as np
import pandas as pd
import math

# --- Dữ liệu đầu vào ---
# Giả định x_coords và y_coords đã tồn tại từ các ô trước
# Ví dụ:
# x_coords = np.array([1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6])
# y_coords = np.array([0.524, -0.084, -0.964, -2.121, -3.558, -5.272, -7.255, -9.494, -11.973])

def create_full_difference_table(y_values):
    """Tạo bảng sai phân đầy đủ (dạng ma trận trên)."""
    num_points = len(y_values)
    table = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points):
        table[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j):
            table[i][j] = table[i+1][j-1] - table[i][j-1]
    return np.array(table)

# Tạo bảng sai phân để dùng cho tính toán
diff_table_for_calc = create_full_difference_table(y_coords)

print("--- Dữ liệu và Bảng sai phân đã sẵn sàng ---")

# (Tùy chọn) Hiển thị bảng cho trực quan
headers = [f'y'] + [f'Δ^{i}y' for i in range(1, len(x_coords))]
df_diff_display = pd.DataFrame(diff_table_for_calc, columns=headers)
center_idx_display = len(x_coords) // 2
# Gán một pandas.Index để tránh cảnh báo/ lỗi từ bộ kiểm tra kiểu tĩnh
df_diff_display.index = pd.Index([f'y_{i - center_idx_display}' for i in range(len(x_coords))])
display(df_diff_display.style.format("{:g}", na_rep=""))

--- Dữ liệu và Bảng sai phân đã sẵn sàng ---


,y,Δ^1y,Δ^2y,Δ^3y,Δ^4y,Δ^5y,Δ^6y,Δ^7y,Δ^8y
y_-4,0.524,-0.608,-0.272,-0.005,0.002,0.004,-0.005,0.006,-0.009
y_-3,-0.084,-0.88,-0.277,-0.003,0.006,-0.001,0.001,-0.003,0
y_-2,-0.964,-1.157,-0.28,0.003,0.005,-4.44089e-15,-0.002,0,0
y_-1,-2.121,-1.437,-0.277,0.008,0.005,-0.002,0,0,0
y_0,-3.558,-1.714,-0.269,0.013,0.003,0,0,0,0
y_1,-5.272,-1.983,-0.256,0.016,0,0,0,0,0
y_2,-7.255,-2.239,-0.24,0,0,0,0,0,0
y_3,-9.494,-2.479,0,0,0,0,0,0,0
y_4,-11.973,0,0,0,0,0,0,0,0


In [4]:
# Giả định x_coords và y_coords đã tồn tại từ Ô 1

# --- Hàm tạo bảng sai phân đầy đủ (dùng cho tính toán) ---
def create_full_difference_table(y_values):
    num_points = len(y_values); table = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points): table[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(num_points - j): table[i][j] = table[i+1][j-1] - table[i][j-1]
    return np.array(table)

# --- Hàm tạo DataFrame hiển thị dạng tam giác dưới ---
def create_lower_diagonal_df(x_values, y_values):
    num_points = len(y_values); center_idx = num_points // 2
    table = [[np.nan for _ in range(num_points)] for _ in range(num_points)]
    for i in range(num_points): table[i][0] = y_values[i]
    for j in range(1, num_points):
        for i in range(j, num_points): table[i][j] = table[i][j-1] - table[i-1][j-1]
    headers = ['y'] + [f'SP{i}' for i in range(1, num_points)]; df = pd.DataFrame(table, columns=headers)
    df.insert(0, 'x', x_values); positions = [f'x_{i - center_idx}' for i in range(num_points)]; df.insert(0, 'Vị trí', positions)
    return df

# Tạo các bảng cần thiết
diff_table_for_calc = create_full_difference_table(y_coords)
df_display = create_lower_diagonal_df(x_coords, y_coords)

# --- Xác định đường đi Sterling, tính vector hệ số và tọa độ tô màu ---
center_idx = len(x_coords) // 2
parameter_vector_coeffs = []
coords_to_highlight = set()
values_to_highlight_raw = []

y0_val = diff_table_for_calc[center_idx, 0]
parameter_vector_coeffs.append(y0_val)
coords_to_highlight.add((center_idx, 'y'))
values_to_highlight_raw.append(y0_val)

for i in range(1, (len(x_coords) // 2) + 1):
    term_order_odd = 2*i - 1
    if term_order_odd < len(x_coords):
        diff1_idx_row = center_idx - i
        diff2_idx_row = center_idx - i + 1
        diff1_val = diff_table_for_calc[diff1_idx_row, term_order_odd]
        diff2_val = diff_table_for_calc[diff2_idx_row, term_order_odd]
        avg_diff_odd = (diff1_val + diff2_val) / 2.0
        coeff_odd = avg_diff_odd / math.factorial(term_order_odd)
        parameter_vector_coeffs.append(coeff_odd)
        values_to_highlight_raw.extend([diff1_val, diff2_val])
        col_name_odd = f'SP{term_order_odd}'
        coords_to_highlight.add((diff1_idx_row + term_order_odd, col_name_odd))
        coords_to_highlight.add((diff2_idx_row + term_order_odd, col_name_odd))

    term_order_even = 2*i
    if term_order_even < len(x_coords):
        diff_idx_row = center_idx - i
        diff_even_val = diff_table_for_calc[diff_idx_row, term_order_even]
        coeff_even = diff_even_val / math.factorial(term_order_even)
        parameter_vector_coeffs.append(coeff_even)
        values_to_highlight_raw.append(diff_even_val)
        col_name_even = f'SP{term_order_even}'
        coords_to_highlight.add((diff_idx_row + term_order_even, col_name_even))

# --- Tạo hàm tô màu (ĐÃ SỬA LỖI) ---
def highlight_sterling_path(data):
    style = 'background-color: yellow; font-weight: bold; color: black;'
    style_df = pd.DataFrame('', index=data.index, columns=data.columns)
    
    for r_display, c_display_name in coords_to_highlight:
        # Kiểm tra tọa độ hợp lệ
        if c_display_name in style_df.columns and r_display in style_df.index:
             val = data.loc[r_display, c_display_name]
             # >>> SỬA LỖI Ở ĐÂY: Chỉ kiểm tra NaN, không kiểm tra isclose(0) <<<
             if not pd.isna(val): # Chỉ cần giá trị không phải NaN là tô màu
                  style_df.loc[r_display, c_display_name] = style
    return style_df

# Áp dụng định dạng và tô màu (không làm tròn)
print(f"\n--- Bảng sai phân (làm nổi bật theo Sterling) ---")
numeric_cols = df_display.select_dtypes(include=np.number).columns
styled_df = df_display.style.apply(highlight_sterling_path, axis=None).format(
    "{:g}", 
    subset=numeric_cols, 
    na_rep=""
)
display(styled_df)

# In vector hệ số Sterling (ĐÃ CHIA GIAI THỪA)
print(f"\n--- Vector hệ số Sterling (Đã chia giai thừa) ---")
# Chuyển thành numpy array để dễ xử lý
vector_coeffs_np = np.array(parameter_vector_coeffs)
print(vector_coeffs_np) # In vector đầy đủ

# >>> THÊM VÀO ĐÂY: In vector chẵn và lẻ <<<
# Lấy các phần tử ở vị trí chẵn (0, 2, 4, ...)
even_coeffs = vector_coeffs_np[::2]
# Lấy các phần tử ở vị trí lẻ (1, 3, 5, ...)
odd_coeffs = vector_coeffs_np[1::2]

print(f"\nChẵn:")
print(even_coeffs)

print(f"\nLẻ:")
print(odd_coeffs)




--- Bảng sai phân (làm nổi bật theo Sterling) ---


,Vị trí,x,y,SP1,SP2,SP3,SP4,SP5,SP6,SP7,SP8
0,x_-4,1,0.524,,,,,,,,
1,x_-3,1.2,-0.084,-0.608,,,,,,,
2,x_-2,1.4,-0.964,-0.88,-0.272,,,,,,
3,x_-1,1.6,-2.121,-1.157,-0.277,-0.005,,,,,
4,x_0,1.8,-3.558,-1.437,-0.28,-0.003,0.002,,,,
5,x_1,2,-5.272,-1.714,-0.277,0.003,0.006,0.004,,,
6,x_2,2.2,-7.255,-1.983,-0.269,0.008,0.005,-0.001,-0.005,,
7,x_3,2.4,-9.494,-2.239,-0.256,0.013,0.005,-4.44089e-15,0.001,0.006,
8,x_4,2.6,-11.973,-2.479,-0.24,0.016,0.003,-0.002,-0.002,-0.003,-0.009



--- Vector hệ số Sterling (Đã chia giai thừa) ---
[-3.55800000e+00 -1.57550000e+00 -1.38500000e-01  9.16666667e-04
  2.08333333e-04 -4.16666667e-06  1.38888889e-06  2.97619048e-07
 -2.23214286e-07]

Chẵn:
[-3.55800000e+00 -1.38500000e-01  2.08333333e-04  1.38888889e-06
 -2.23214286e-07]

Lẻ:
[-1.57550000e+00  9.16666667e-04 -4.16666667e-06  2.97619048e-07]


In [5]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# even_coeffs: Vector hệ số ở vị trí chẵn (đã chia giai thừa)

# --- Bước 1: Chuẩn bị dữ liệu ---
num_even_coeffs = len(even_coeffs)
num_rows_product_table = num_even_coeffs - 1

# --- Bước 2: Tạo Bảng Tích ---

# Khởi tạo bảng với NaN
product_table_data = np.full((num_rows_product_table + 1, num_even_coeffs), np.nan)

# Hàng đầu tiên (hàng 0) là [0, 0, ..., 1]
product_table_data[0, :] = 0.0
product_table_data[0, -1] = 1.0

# Danh sách các giá trị c = k^2 (k bắt đầu từ 0)
c_values = [(i**2) for i in range(num_rows_product_table)]

# Lặp để tính các hàng tiếp theo
# a: là hàng trước đó
# b: là hàng đang tính
for row_idx in range(num_rows_product_table):
    c = c_values[row_idx]
    a = product_table_data[row_idx] # Lấy hàng đã tính ở bước trước
    b = np.zeros(num_even_coeffs)   # Hàng mới

    # >>> Áp dụng công thức ĐÚNG: b_k = a_{k+1} - a_k * c <<<
    for k in range(num_even_coeffs):
        # Lấy a[k+1], nếu k là chỉ số cuối cùng thì coi như là 0
        a_k_plus_1 = a[k+1] if k < num_even_coeffs - 1 else 0.0 
        a_k = a[k]
        b[k] = a_k_plus_1 - (a_k * c) # Tính giá trị b_k
        
    # Lưu hàng mới vào bảng
    product_table_data[row_idx + 1] = b

# --- Bước 3: Tạo và hiển thị DataFrame ---

# Tạo header cho các cột
product_headers = [f'Col_{i}' for i in range(num_even_coeffs)]

# Tạo index cho các hàng (bao gồm hàng khởi tạo và các giá trị c=k^2)
product_index = ['Khởi tạo (a)'] + c_values

# Tạo DataFrame
df_product = pd.DataFrame(product_table_data, index=product_index, columns=product_headers)
df_product.index.name = "Giá trị c (k^2)"

# Định dạng hiển thị (không làm tròn)
styled_product_df = df_product.style.format("{:g}", na_rep="")

print("\n--- Bảng Tích (Tính theo công thức b_k = a_{k+1} - a_k * c) ---")
display(styled_product_df)


--- Bảng Tích (Tính theo công thức b_k = a_{k+1} - a_k * c) ---


,Col_0,Col_1,Col_2,Col_3,Col_4
Giá trị c (k^2),,,,,
Khởi tạo (a),0,0,0,0,1
0,0,0,0,1,0
1,0,0,1,-1,0
4,0,1,-5,4,0
9,1,-14,49,-36,0


In [6]:
# Giả định các biến sau đã tồn tại từ các ô trước:
# even_coeffs: Vector hệ số chẵn ban đầu (đã chia giai thừa)
# odd_coeffs: Vector hệ số lẻ ban đầu (đã chia giai thừa)
# product_table_data: Ma trận NumPy chứa dữ liệu của Bảng Tích

# --- B1: In lại vector tham số chẵn và lẻ ban đầu ---
print("--- B1: Vector tham số chẵn và lẻ ban đầu ---")
print("Vector chẵn (even_coeffs):")
print(even_coeffs)
print("\nVector lẻ (odd_coeffs):")
print(odd_coeffs)
print("-" * 50)


# --- B2: In ra ma trận để nhân với vector tham số chẵn ---
matrix_for_even = product_table_data
print("--- B2: Ma trận để nhân với vector chẵn (Toàn bộ Bảng Tích) ---")
with np.printoptions(precision=15, suppress=True, threshold=np.inf):
    print(matrix_for_even)
print("-" * 50)


# --- B3: In ra ma trận để nhân với vector tham số lẻ ---
matrix_for_odd = None
try:
    matrix_for_odd = product_table_data[1:, :-1]
    print("--- B3: Ma trận để nhân với vector lẻ (Bảng Tích bỏ hàng 0, cột cuối) ---")
    with np.printoptions(precision=15, suppress=True, threshold=np.inf):
        print(matrix_for_odd)
except IndexError as e:
    print(f"--- B3: Lỗi khi cắt ma trận cho vector lẻ: {e} ---")
print("-" * 50)


# --- B4: Thực hiện phép nhân và GÁN KẾT QUẢ ---
print("--- B4: Thực hiện phép nhân ---")
new_even_coeffs = None # Khởi tạo là None
new_odd_coeffs = None  # Khởi tạo là None

# Phép nhân cho vector chẵn
if matrix_for_even is not None and even_coeffs is not None:
    print("Tính vector chẵn mới:")
    try:
        print(f"Thực hiện phép nhân: even_coeffs @ matrix_for_even")
        new_even_coeffs_calc = even_coeffs @ matrix_for_even
        
        # >>> THAY ĐỔI Ở ĐÂY: Gán trực tiếp, KHÔNG đảo ngược <<<
        new_even_coeffs = new_even_coeffs_calc 
        
        print("Kết quả vector chẵn mới (KHÔNG đảo ngược):")
        # In kết quả cuối cùng (không đảo ngược)
        print("[", end="")
        for i, val in enumerate(new_even_coeffs):
            print(f"{val}", end="")
            if i < len(new_even_coeffs) - 1: print(", ", end="")
        print("]")
        
    except ValueError as e:
        print(f"Lỗi khi nhân ma trận cho vector chẵn: {e}.")
else:
    print("Không thể tính vector chẵn mới.")

print("-" * 30)

# Phép nhân cho vector lẻ (giữ nguyên)
if matrix_for_odd is not None and odd_coeffs is not None:
    print("Tính vector lẻ mới:")
    try:
        print(f"Thực hiện phép nhân: matrix_for_odd.T @ odd_coeffs")
        new_odd_coeffs = matrix_for_odd.T @ odd_coeffs
        
        print("Kết quả vector lẻ mới:")
        print("[", end="")
        for i, val in enumerate(new_odd_coeffs):
            print(f"{val}", end="")
            if i < len(new_odd_coeffs) - 1: print(", ", end="")
        print("]")

    except ValueError as e:
        print(f"Lỗi khi nhân ma trận cho vector lẻ: {e}.")
else:
    print("Không thể tính vector lẻ mới.")

print("=" * 50)
print("--- KẾT THÚC Ô TÍNH TOÁN ---")

--- B1: Vector tham số chẵn và lẻ ban đầu ---
Vector chẵn (even_coeffs):
[-3.55800000e+00 -1.38500000e-01  2.08333333e-04  1.38888889e-06
 -2.23214286e-07]

Vector lẻ (odd_coeffs):
[-1.57550000e+00  9.16666667e-04 -4.16666667e-06  2.97619048e-07]
--------------------------------------------------
--- B2: Ma trận để nhân với vector chẵn (Toàn bộ Bảng Tích) ---
[[  0.   0.   0.   0.   1.]
 [  0.   0.   0.   1.   0.]
 [  0.   0.   1.  -1.   0.]
 [  0.   1.  -5.   4.   0.]
 [  1. -14.  49. -36.   0.]]
--------------------------------------------------
--- B3: Ma trận để nhân với vector lẻ (Bảng Tích bỏ hàng 0, cột cuối) ---
[[  0.   0.   0.   1.]
 [  0.   0.   1.  -1.]
 [  0.   1.  -5.   4.]
 [  1. -14.  49. -36.]]
--------------------------------------------------
--- B4: Thực hiện phép nhân ---
Tính vector chẵn mới:
Thực hiện phép nhân: even_coeffs @ matrix_for_even
Kết quả vector chẵn mới (KHÔNG đảo ngược):
[-2.2321428571363334e-07, 4.513888888869271e-06, 0.00019045138888906134, -0.1386

In [7]:
import numpy as np # Đảm bảo numpy đã được import

def format_polynomial_descending(coeffs_asc):
    """Hàm định dạng đa thức, hiển thị tất cả các số hạng không bằng 0,
       sắp xếp theo bậc giảm dần và hiển thị đầy đủ chữ số.
    Args:
        coeffs_asc (list or np.array): Danh sách hệ số theo bậc tăng dần (a0, a1, ...).
    Returns:
        str: Chuỗi biểu diễn đa thức.
    """
    # Đảo ngược hệ số để có thứ tự bậc giảm dần (an, an-1, ...)
    coeffs_desc = coeffs_asc[::-1]
    poly_str = ""
    n = len(coeffs_desc) - 1 # Bậc cao nhất

    for i, coeff in enumerate(coeffs_desc):
        # Chỉ xử lý các hệ số khác 0
        if coeff == 0.0:
            continue

        power = n - i # Bậc của số hạng hiện tại

        # --- Xử lý dấu ---
        sign = " - " if coeff < 0 else " + "
        coeff_abs = abs(coeff)

        # Nếu là số hạng đầu tiên, bỏ dấu '+'
        if poly_str == "":
            sign = "" if coeff > 0 else "-"

        # --- Xử lý hệ số ---
        # Bỏ hệ số 1 nếu bậc > 0
        if np.isclose(coeff_abs, 1) and power != 0:
            coeff_str = ""
        else:
            # Sử dụng str() để hiển thị đầy đủ, không dùng format 'g' nữa
            coeff_str = str(coeff_abs)

        # --- Xử lý biến x ---
        if power == 1:
            var_str = "t"
        elif power == 0:
            # Nếu là hằng số (bậc 0), luôn hiển thị hệ số
            var_str = ""
            coeff_str = str(coeff_abs) # Đảm bảo hằng số được hiển thị
        else:
            var_str = f"t^{power}"

        # --- Kết hợp ---
        separator = "*" if coeff_str != "" and var_str != "" else ""
        poly_str += f"{sign}{coeff_str}{separator}{var_str}"

    # Xử lý trường hợp đa thức bằng 0
    if poly_str == "":
        return "0"
    else:
        # Xóa dấu cách thừa ở đầu nếu có
        return poly_str.strip()

print("Hàm format_polynomial_descending đã được định nghĩa.")

Hàm format_polynomial_descending đã được định nghĩa.


In [8]:
# Giả định các biến sau đã tồn tại từ Ô TRÊN:
# new_even_coeffs: Vector hệ số chẵn mới (đã tính, BẬC GIẢM DẦN)
# new_odd_coeffs: Vector hệ số lẻ mới (đã tính, BẬC GIẢM DẦN)
# even_coeffs: Vector hệ số chẵn ban đầu (để lấy độ dài)
# odd_coeffs: Vector hệ số lẻ ban đầu (để lấy độ dài)
# format_polynomial_descending: Hàm định dạng đa thức (đã định nghĩa ở ô trước nữa)

print("--- B5: Kết hợp vector hệ số ---")
print("Sử dụng các vector mới đã tính:")
if new_even_coeffs is not None:
    with np.printoptions(threshold=np.inf): print("Vector chẵn mới:", new_even_coeffs)
else: print("Vector chẵn mới: Lỗi!")
if new_odd_coeffs is not None:
    with np.printoptions(threshold=np.inf): print("Vector lẻ mới:", new_odd_coeffs)
else: print("Vector lẻ mới: Lỗi!")
print("-" * 30)


# Chỉ thực hiện kết hợp nếu cả hai vector mới đều hợp lệ
if new_even_coeffs is not None and new_odd_coeffs is not None:

    # Xác định độ dài và bậc dựa trên vector ban đầu
    len_even_orig = len(even_coeffs)
    len_odd_orig = len(odd_coeffs)
    total_coeffs = len_even_orig + len_odd_orig
    final_coeffs_descending = np.zeros(total_coeffs)

    even_idx = 0
    odd_idx = 0

    # Lặp từ đầu (bậc cao) đến cuối (bậc thấp) của final_coeffs_descending
    for i in range(total_coeffs):
        power = total_coeffs - 1 - i # Bậc hiện tại
        if power % 2 == 0: # Bậc chẵn
            if even_idx < len(new_even_coeffs):
                final_coeffs_descending[i] = new_even_coeffs[even_idx]
                even_idx += 1
        else: # Bậc lẻ
            if odd_idx < len(new_odd_coeffs):
                final_coeffs_descending[i] = new_odd_coeffs[odd_idx]
                odd_idx += 1

    # --- In kết quả ---
    print("--- Vector hệ số cuối cùng của Đa thức Nội suy (ĐTNS) ---")
    print("(Sắp xếp theo bậc giảm dần)")
    with np.printoptions(threshold=np.inf):
        print(final_coeffs_descending)

    # In ra đa thức dạng chuỗi
    try:
        final_coeffs_ascending = final_coeffs_descending[::-1]
        print("\n--- Đa thức nội suy P(x) hoàn chỉnh ---")
        print(f"P(x) =", format_polynomial_descending(final_coeffs_ascending))
    except NameError:
        print("\n(Hàm format_polynomial_descending chưa được định nghĩa để in đa thức)")

else:
    print("LỖI: Không thể kết hợp vector hệ số do lỗi trong các bước tính toán trước đó.")

print("=" * 50)
print("--- KẾT THÚC ---")

--- B5: Kết hợp vector hệ số ---
Sử dụng các vector mới đã tính:
Vector chẵn mới: [-2.23214286e-07  4.51388889e-06  1.90451389e-04 -1.38694742e-01
 -3.55800000e+00]
Vector lẻ mới: [ 2.97619048e-07 -8.33333333e-06  9.52083333e-04 -1.57644405e+00]
------------------------------
--- Vector hệ số cuối cùng của Đa thức Nội suy (ĐTNS) ---
(Sắp xếp theo bậc giảm dần)
[-2.23214286e-07  2.97619048e-07  4.51388889e-06 -8.33333333e-06
  1.90451389e-04  9.52083333e-04 -1.38694742e-01 -1.57644405e+00
 -3.55800000e+00]

--- Đa thức nội suy P(x) hoàn chỉnh ---
P(x) = -2.2321428571363334e-07*t^8 + 2.9761904761930123e-07*t^7 + 4.513888888869271e-06*t^6 - 8.33333333334244e-06*t^5 + 0.00019045138888906134*t^4 + 0.0009520833333334205*t^3 - 0.1386947420634925*t^2 - 1.576444047619048*t - 3.558
--- KẾT THÚC ---


## Đoạn code tính các giá trị của P(x) tại x = x' 


In [9]:
# Giả định các biến x_coords và diff_table_for_calc đã có từ các ô trước

def get_gauss_1_coeffs_in_t(x_data, full_diff_table):
    """
    Tính các hệ số của đa thức P(t) trực tiếp từ công thức Gauss I.
    """
    n = len(x_data) - 1
    center_idx = n // 2
    
    # Bắt đầu với số hạng đầu tiên của công thức: y_0
    final_poly_in_t = np.array([full_diff_table[center_idx][0]])
    
    # Đa thức tích (basis polynomial), ban đầu là 1
    product_poly_in_t = np.array([1.0])
    
    # Lặp qua từng số hạng tiếp theo trong công thức
    for i in range(1, n + 1):
        
        # Xây dựng đa thức tích P_i(t)
        if i == 1:
            factor_poly = np.array([0.0, 1.0])  # t
        elif i % 2 == 0:
            k = i / 2
            factor_poly = np.array([-k, 1.0])  # (t - k)
        else:
            k = (i - 1) / 2
            factor_poly = np.array([k, 1.0])   # (t + k)
        product_poly_in_t = P.polymul(product_poly_in_t, factor_poly)
        
        # Lấy giá trị sai phân tương ứng
        diff_row_idx = center_idx - (i // 2)
        diff_val = full_diff_table[diff_row_idx][i]
        
        # Tạo đa thức cho toàn bộ số hạng
        term_poly_in_t = (diff_val / math.factorial(i)) * product_poly_in_t
        
        # Cộng đa thức của số hạng này vào kết quả cuối cùng
        final_poly_in_t = P.polyadd(final_poly_in_t, term_poly_in_t)
        
    return final_poly_in_t

# --- Thực thi và định nghĩa biến coeffs_in_t ---
coeffs_in_t = get_gauss_1_coeffs_in_t(x_coords, diff_table_for_calc)

# In kết quả để xác nhận
print("--- Biến 'coeffs_in_t' đã được định nghĩa ---")
print(coeffs_in_t)

--- Biến 'coeffs_in_t' đã được định nghĩa ---
[-3.55800000e+00 -1.57644405e+00 -1.38694742e-01  9.52083333e-04
  1.90451389e-04 -8.33333333e-06  4.51388889e-06  2.97619048e-07
 -2.23214286e-07]


In [11]:
import math
import numpy as np
import pandas as pd

# Thiết lập độ chính xác hiển thị cho Pandas
pd.set_option('display.precision', 15)

# --- KIỂM TRA BIẾN ĐẦU VÀO ---
try:
    # Kiểm tra xem x_coords và coeffs_in_t đã được định nghĩa chưa
    _ = x_coords 
    _ = coeffs_in_t
    print("Các biến 'x_coords' và 'coeffs_in_t' (Gauss I) đã tồn tại.")
except NameError as e:
    print(f"LỖI: Biến cần thiết chưa được định nghĩa: {e}")
    raise # Dừng hẳn

# --- 1. Chuẩn bị dữ liệu và chuyển đổi x -> t (ĐÃ SỬA) ---
p_coeffs_descending = coeffs_in_t[::-1] 
x_input = 1.43 # Bạn có thể thay đổi giá trị x ở đây
n = len(x_coords) - 1
center_idx = n // 2
x0 = x_coords[center_idx]
h = x_coords[1] - x_coords[0]

# >>> SỬA LỖI Ở ĐÂY: Bỏ "- 0.5" để tính đúng giá trị t cho Gauss I <<<
t_val = (x_input - x0) / h 

print("\n--- Bước 1: Chuyển đổi X sang T ---")
print(f"Giá trị x đầu vào: x = {x_input}")
print(f"Mốc trung tâm: x0 = {x0}, bước nhảy h = {h}")
# In công thức đúng
print(f"Giá trị t tương ứng (Gauss I): t = ({x_input} - {x0}) / {h} = {t_val}")


# --- 2. Hàm tính đạo hàm bằng Horner (giữ nguyên) ---
def tinh_bang_dao_ham_horner(parameter_vector_coeffs_desc, t_value_to_eval):
    # ... (code hàm giữ nguyên như bạn đã cung cấp) ...
    print("\n" + "="*50)
    print(f"Đa thức P(t) có hệ số (bậc giảm dần): {parameter_vector_coeffs_desc}")
    print(f"Tính các đạo hàm tại t = {t_value_to_eval}")
    current_coeffs = list(parameter_vector_coeffs_desc); n = len(current_coeffs) - 1
    summary_data = []; k_factorial = 1.0
    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- BƯỚC {k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---") # Sửa lại {k+2}.{k+1} thành {k+1}
        m = len(current_coeffs); deg = m - 1
        header = [f'a_{deg-i}' for i in range(deg + 1)]; row_coeffs = current_coeffs
        row_products = ['']; row_results = []
        b = current_coeffs[0]; row_results.append(b)
        quotient_coeffs = [b]
        for i in range(1, m):
            product = b * t_value_to_eval; row_products.append(product)
            b = product + current_coeffs[i]; row_results.append(b)
            if i < m -1: quotient_coeffs.append(b)
        remainder = b
        df = pd.DataFrame([row_coeffs, row_products, row_results],
                          index=['Hệ số vào', f'Nhân với t={t_value_to_eval}', 'Kết quả'],
                          columns=header)
        display(df)
        if k > 0: k_factorial *= k
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(t)", derivative_value])
        print(f"-> Số dư R_{k+1} = {remainder}")
        print(f"-> Giá trị đạo hàm P^({k})(t) = R_{k+1} * {k}! = {remainder} * {k_factorial} = {derivative_value}")
        current_coeffs = quotient_coeffs
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm", f"Giá trị tại t = {t_value_to_eval}"])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_input} ---")
    display(summary_df)
    
    # Trả về bảng tổng hợp để bước 4 sử dụng
    return summary_df

# --- 3. Thực thi hàm ---
summary_df_t = tinh_bang_dao_ham_horner(p_coeffs_descending, t_val)

# --- 4. Chuyển đổi đạo hàm từ P(t) sang P(x) (giữ nguyên) ---
print("\n" + "="*50)
print("--- CHUYỂN ĐỔI SANG ĐẠO HÀM THEO x ---")
print(f"Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / dt^k)")

derivatives_wrt_x = []
for k in range(len(summary_df_t)):
     dkP_dtk = summary_df_t.iloc[k, 1] 
     dkP_dxk = (1 / (h**k)) * dkP_dtk
     derivatives_wrt_x.append([f"P^({k})(x')", dkP_dxk])

column_name_x = f"Giá trị tại x = {x_input}"
summary_df_x = pd.DataFrame(derivatives_wrt_x, columns=["Đạo hàm (theo x)", column_name_x])
print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_input} (theo x) ---")
display(summary_df_x.style.format("{:g}", subset=[column_name_x]))

Các biến 'x_coords' và 'coeffs_in_t' (Gauss I) đã tồn tại.

--- Bước 1: Chuyển đổi X sang T ---
Giá trị x đầu vào: x = 1.43
Mốc trung tâm: x0 = 1.8, bước nhảy h = 0.19999999999999996
Giá trị t tương ứng (Gauss I): t = (1.43 - 1.8) / 0.19999999999999996 = -1.850000000000001

Đa thức P(t) có hệ số (bậc giảm dần): [-2.23214286e-07  2.97619048e-07  4.51388889e-06 -8.33333333e-06
  1.90451389e-04  9.52083333e-04 -1.38694742e-01 -1.57644405e+00
 -3.55800000e+00]
Tính các đạo hàm tại t = -1.850000000000001

--- BƯỚC 1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_8,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-0.000000223214286,0.000000297619048,0.000004513888889,-0.000008333333333,0.000190451388889,0.000952083333333,-0.138694742063492,-1.576444047619048,-3.558000000000000
Nhân với t=-1.850000000000001,,0.000000412946429,-0.000001314546131,-0.000005918784102,0.000026366417256,-0.000401112941368,-0.001019295225137,0.258470968983964,2.438250195474907
Kết quả,-0.000000223214286,0.000000710565476,0.000003199342758,-0.000014252117435,0.000216817806145,0.000550970391966,-0.139714037288629,-1.317973078635084,-1.119749804525093


-> Số dư R_1 = -1.1197498045250929
-> Giá trị đạo hàm P^(0)(t) = R_1 * 0! = -1.1197498045250929 * 1.0 = -1.1197498045250929

--- BƯỚC 2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_7,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,-0.000000223214286,0.000000710565476,0.000003199342758,-0.000014252117435,0.000216817806145,0.000550970391966,-0.139714037288629,-1.317973078635084
Nhân với t=-1.850000000000001,,0.000000412946429,-0.000002078497024,-0.000002073564608,0.000030202511781,-0.000456987588162,-0.000173868187037,0.258792625129982
Kết quả,-0.000000223214286,0.000001123511905,0.000001120845734,-0.000016325682044,0.000247020317925,0.000093982803804,-0.139887905475666,-1.059180453505102


-> Số dư R_2 = -1.059180453505102
-> Giá trị đạo hàm P^(1)(t) = R_2 * 1! = -1.059180453505102 * 1.0 = -1.059180453505102

--- BƯỚC 3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


...

,a_0
Hệ số vào,-0.000000223214286
Nhân với t=-1.850000000000001,
Kết quả,-0.000000223214286


-> Số dư R_9 = -2.2321428571363334e-07
-> Giá trị đạo hàm P^(8)(t) = R_9 * 8! = -2.2321428571363334e-07 * 40320.0 = -0.008999999999973696

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 1.43 ---


,Đạo hàm,Giá trị tại t = -1.850000000000001
0,P^(0)(t),-1.119749804525093
1,P^(1)(t),-1.059180453505102
2,P^(2)(t),-0.278266289051918
3,P^(3)(t),-0.005527078742186
4,P^(4)(t),0.006006924739577
5,P^(5)(t),0.005051812499999
6,P^(6)(t),-0.014926249999971
7,P^(7)(t),0.018149999999953
8,P^(8)(t),-0.008999999999974



--- CHUYỂN ĐỔI SANG ĐẠO HÀM THEO x ---
Sử dụng công thức: d^k P / dx^k = (1 / h^k) * (d^k P / dt^k)

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 1.43 (theo x) ---


,Đạo hàm (theo x),Giá trị tại x = 1.43
0,P^(0)(x'),-1.11975
1,P^(1)(x'),-5.2959
2,P^(2)(x'),-6.95666
3,P^(3)(x'),-0.690885
4,P^(4)(x'),3.75433
5,P^(5)(x'),15.7869
6,P^(6)(x'),-233.223
7,P^(7)(x'),1417.97
8,P^(8)(x'),-3515.62
